### Imports

In [15]:
import pandas as pd
import numpy as np
from scipy.stats import invgamma, multivariate_normal

### Load the Data

In [4]:
mazda_df = pd.read_csv('2_Mazda2.csv')
mazda_df['kms'] = mazda_df['kms']/1000
mazda_df.head()

,kms,bueno,years,lnprecio,mecanico,precio
0,131.0,0,9,16.951005,1,23000000
1,107.0,0,9,16.796129,1,19700000
2,139.0,0,9,16.883563,1,21500000
3,98.0,1,9,16.901997,0,21900000
4,74.0,0,8,16.929026,1,22500000


In [10]:
feature_matrix = mazda_df[['kms', 'bueno', 'years', 'mecanico']].values
feature_matrix = np.hstack((np.ones((feature_matrix.shape[0], 1)), feature_matrix))
dependent_variable = mazda_df['lnprecio'].values.reshape(-1, 1)
n_features = feature_matrix.shape[1]
n_samples = feature_matrix.shape[0]

### Normal-Inverse Gamma Model with Independent Priors

#### Gibbs Sampler

In [19]:
# Hyperparameters
prior_covariance_matrix = 1000 * np.eye(n_features)
prior_inv_covariance_matrix = np.linalg.inv(prior_covariance_matrix)
prior_mean = np.zeros((n_features, 1))
prior_shape_parameter = 0.001
prior_rate_parameter = 0.001

# Define parameters
burn_first = 5000
iters = 10000
total_iters = burn_first + iters
gibbs_mean_matrix = np.zeros((total_iters, n_features))
gibbs_variance_matrix = np.ones((total_iters, 1))

# Calculate posterior parameters
first_sigma_squared = 10
pth_covariance_matrix = np.linalg.inv(prior_inv_covariance_matrix + np.dot(feature_matrix.T, feature_matrix) / first_sigma_squared)
pth_mean_vector = np.dot(pth_covariance_matrix, np.dot(prior_inv_covariance_matrix, prior_mean) + np.dot(feature_matrix.T, dependent_variable) / first_sigma_squared)
pth_shape_parameter = prior_shape_parameter + n_samples

# Gibbs sampling
for i in range(total_iters):
    # Sample and calculate posterior parameters
    mean_gibbs = multivariate_normal.rvs(pth_mean_vector.reshape(-1), pth_covariance_matrix)
    aux_rate_parameter = dependent_variable - np.dot(feature_matrix, mean_gibbs.reshape(-1, 1))
    pth_rate_parameter = prior_rate_parameter + np.dot(aux_rate_parameter.T, aux_rate_parameter)
    sigma_squared_gibbs = invgamma.rvs(pth_shape_parameter / 2, pth_rate_parameter / 2)

    # Update posterior parameters
    pth_covariance_matrix = np.linalg.inv(prior_inv_covariance_matrix + np.dot(feature_matrix.T, feature_matrix) / sigma_squared_gibbs)
    pth_mean_vector = np.dot(pth_covariance_matrix, np.dot(prior_inv_covariance_matrix, prior_mean) + np.dot(feature_matrix.T, dependent_variable) / sigma_squared_gibbs)

    # Save samples
    gibbs_mean_matrix[i, :] = mean_gibbs
    gibbs_variance_matrix[i, :] = sigma_squared_gibbs

#### Markov Chain Monte Carlo Sampling